## Limpieza del Dataset

In [31]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

#### PASO 1: Cargar el Dataset

In [32]:
#En este paso, cargamos el archivo CSV con los datos de tráfico de red. 

df = pd.read_csv("archive/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")


##### Detalles del dataset
Estamos cargando un CSV que contiene una lista de logs en los que hay posibles ataques DDos

In [33]:
# También mostramos las primeras filas del dataset para entender y previsualizar su contenido.
print("\n### Primeras filas del dataset:")
print(df.head())


### Primeras filas del dataset:
    Destination Port   Flow Duration   Total Fwd Packets  \
0              54865               3                   2   
1              55054             109                   1   
2              55055              52                   1   
3              46236              34                   1   
4              54863               3                   2   

    Total Backward Packets  Total Length of Fwd Packets  \
0                        0                           12   
1                        1                            6   
2                        1                            6   
3                        1                            6   
4                        0                           12   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                             0                       6   
1                             6                       6   
2                             6                       6   
3              

# Analisis de las Columnas del Dataset CIC-IDS2017

Este dataset contiene múltiples columnas con información detallada sobre el tráfico de red. A continuación, se describen las más relevantes:

## 🔹 1. Información sobre el Flujo de Red  
- **Destination Port**: Puerto de destino del tráfico.  
- **Flow Duration**: Duración total del flujo en milisegundos.  
- **Total Fwd Packets**: Número total de paquetes enviados en dirección forward.  
- **Total Backward Packets**: Número total de paquetes enviados en dirección backward.  

## 🔹 2. Características de los Paquetes  
- **Total Length of Fwd Packets**: Longitud total de los paquetes enviados en dirección forward.  
- **Total Length of Bwd Packets**: Longitud total de los paquetes enviados en dirección backward.  
- **Fwd Packet Length Max / Min / Mean / Std**: Medidas estadísticas de la longitud de los paquetes enviados en forward.  
- **Bwd Packet Length Max / Min / Mean / Std**: Medidas estadísticas de la longitud de los paquetes enviados en backward.  

## 🔹 3. Estadísticas de Tasa de Flujo  
- **Flow Bytes/s**: Número de bytes transmitidos por segundo en el flujo.  
- **Flow Packets/s**: Número de paquetes transmitidos por segundo.  
- **Flow IAT Mean / Max / Min / Std**: Intervalo de tiempo promedio, máximo, mínimo y desviación estándar entre paquetes en el flujo.  

## 🔹 4. Información sobre Flags y Señales de Control  
- **SYN Flag Count / FIN Flag Count / RST Flag Count**: Contadores de los flags TCP utilizados en el flujo.  
- **PSH Flag Count / ACK Flag Count / URG Flag Count**: Contadores de otros flags de control TCP.  

## 🔹 5. Información sobre la Ventana TCP  
- **Init_Win_bytes_forward**: Tamaño de la ventana TCP en la dirección forward.  
- **Init_Win_bytes_backward**: Tamaño de la ventana TCP en la dirección backward.  

## 🔹 6. Estado del Tráfico (Label)  
- **Label**: Indica si el tráfico es `BENIGN` (normal) o pertenece a un ataque específico como `DDoS`, `PortScan`, etc.  

---
**Estas características se utilizarán para entrenar un modelo de IA que pueda detectar patrones de tráfico malicioso en redes.**


In [34]:
# Revisamos cuántos valores nulos hay en cada columna.
print("\n### Valores nulos en el dataset:")
print(df.isnull().sum()[df.isnull().sum() > 0]) 



### Valores nulos en el dataset:
Flow Bytes/s    4
dtype: int64


**Interpretacion de resultados:**
Vemos que solo hay 4 valores nulos en el dataset, por tanto como son pocos valores nulos vamos a eliminarlos del dataset. Si fuesen muchos valores Nan podríamos reemplazarlos por la media de la columna para no borrar mucha información.

In [35]:
# Eliminamos las filas que contienen valores nulos.
df.dropna(inplace=True)

#comprobamos que no haya valores nulos
print("\n### Valores nulos en el dataset:")
print(df.isnull().sum()[df.isnull().sum() > 0])

df.columns = df.columns.str.strip()  # Elimina espacios en los nombres de columnas





### Valores nulos en el dataset:
Series([], dtype: int64)


# Eliminación de Datos Duplicados

En el preprocesamiento de datos, es fundamental eliminar los registros duplicados, ya que pueden afectar el rendimiento del modelo de aprendizaje automático. Los duplicados pueden surgir debido a errores en la recopilación de datos o repeticiones en la generación del dataset.

## ¿Por qué eliminamos los duplicados?

- **Evita sesgos en el entrenamiento**: Si una clase tiene más registros duplicados, el modelo puede sobreajustarse a esos datos y generalizar mal en nuevos casos.
- **Optimiza el uso de recursos**: Trabajar con datos redundantes aumenta el consumo de memoria y tiempo de cómputo sin aportar nueva información útil.
- **Mejora la calidad del dataset**: Un dataset sin duplicados garantiza que cada muestra contribuye con información única al modelo, favoreciendo su capacidad de aprendizaje.

En este proceso, identificamos y eliminamos registros duplicados utilizando la función `drop_duplicates()`, asegurando que el dataset final contenga únicamente datos relevantes y sin repeticiones innecesarias.


In [36]:
# ## 4️⃣ Eliminación de Datos Duplicados
# Algunos registros pueden estar duplicados, lo que afectaría el análisis.
print("\n### Cantidad de datos duplicados antes de eliminarlos:", df.duplicated().sum())
df.drop_duplicates(inplace=True)
print("Cantidad de datos duplicados después de eliminarlos:", df.duplicated().sum())



### Cantidad de datos duplicados antes de eliminarlos: 2633
Cantidad de datos duplicados después de eliminarlos: 0


# Conversión de Datos Categóricos

En los modelos de aprendizaje automático, es necesario que todas las variables sean numéricas, ya que la mayoría de los algoritmos no pueden procesar datos en formato de texto. En este dataset, la columna `Label` indica si el tráfico es benigno o corresponde a un ataque, pero está representada como valores categóricos en texto (`BENIGN`, `DDoS`, etc.).

## ¿Por qué convertimos los datos categóricos?

- **Compatibilidad con modelos de IA**: La mayoría de los algoritmos de aprendizaje automático solo funcionan con datos numéricos.
- **Estandarización del dataset**: Facilita la comparación entre distintas clases dentro del modelo.
- **Optimización del procesamiento**: Representar los valores categóricos como números reduce el tiempo de cómputo.

Para esta conversión, se utiliza `LabelEncoder()`, que asigna un número a cada categoría, permitiendo que el modelo pueda interpretar correctamente la variable `Label` sin perder información.


In [37]:
# ## 5️⃣ Conversión de Datos Categóricos
# La columna 'Label' indica si el tráfico es benigno o un ataque. Convertimos los valores de texto a números.
df['Label'] = df['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)


#Comprobamos que se haya hecho la conversión
print("\n### Columna 'Label' después de la conversión:")
print(df['Label'].value_counts())


### Columna 'Label' después de la conversión:
Label
1    128016
0     95092
Name: count, dtype: int64


# Normalización de Datos Numéricos

En el preprocesamiento de datos, es fundamental escalar las variables numéricas para mejorar el rendimiento del modelo de aprendizaje automático. La normalización ayuda a que los modelos sean más estables y precisos, especialmente en algoritmos sensibles a la escala de los datos, como redes neuronales y SVM.

## ¿Por qué normalizamos los datos?

- **Diferentes escalas pueden afectar el modelo**: Algunas características tienen valores muy grandes (ej. `Flow Bytes/s`), mientras que otras tienen valores pequeños (`Packet Length`). Si no se normalizan, el modelo podría dar más importancia a ciertas variables solo por su magnitud.
- **Mejora la convergencia del entrenamiento**: Los modelos basados en gradiente (como redes neuronales) entrenan más rápido y de manera más estable con datos normalizados.
- **Evita sesgos en la clasificación**: Al escalar todas las variables a una misma escala, evitamos que unas características dominen sobre otras.

## ¿Cómo lo hacemos?

Utilizamos `StandardScaler()` de `sklearn.preprocessing`, que transforma los datos para que tengan una media de 0 y desviación estándar de 1. Esto se aplica a todas las columnas numéricas del dataset.

Después de este paso, los datos estarán listos para ser utilizados en el modelo de IA sin riesgo de que la escala de los valores afecte negativamente el entrenamiento.


In [38]:
# ## 6️⃣ Normalización de Datos Numéricos

# Reemplazamos valores infinitos con la media de la columna
df.replace([float('inf'), float('-inf')], pd.NA, inplace=True)
df.fillna(df.mean(), inplace=True)

# Verificamos si aún quedan valores infinitos o nulos después del reemplazo
print("\n### Valores nulos después de corregir infinitos:")
print(df.isnull().sum().sum())  # Debería ser 0

# Aplicamos StandardScaler para normalizar los datos numéricos
scaler = StandardScaler()
columnas_numericas = df.select_dtypes(include=['float64', 'int64']).columns
df[columnas_numericas] = scaler.fit_transform(df[columnas_numericas])

print("\n### Normalización completada correctamente.")


/tmp/ipykernel_12169/2220213856.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(df.mean(), inplace=True)



### Valores nulos después de corregir infinitos:
0

### Normalización completada correctamente.


In [39]:
# ## 7️⃣ Guardar Dataset Limpio
# Finalmente, guardamos el dataset limpio en un nuevo archivo CSV.
cleaned_file_path = "archive/cleaned_dataset.csv"
df.to_csv(cleaned_file_path, index=False)
print(f"\n### Dataset limpio guardado en: {cleaned_file_path}")



### Dataset limpio guardado en: archive/cleaned_dataset.csv
